In [181]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
import plotly.figure_factory as ff

import sqlite3
import requests

from functools import reduce

In [182]:
# Importing storm events data from database
conn = sqlite3.connect('../data/raw/storms.db')
c = conn.cursor()

query = '''
          SELECT BEGIN_YEARMONTH AS YEARMONTH,
                 BEGIN_DAY AS DAY,
                 CZ_NAME AS PARISH,
                 EVENT_TYPE,
                 BEGIN_LAT AS LAT,
                 BEGIN_LON AS LON,
                 INJURIES_DIRECT,
                 INJURIES_INDIRECT,
                 DEATHS_DIRECT,
                 DEATHS_INDIRECT,
                 DAMAGE_PROPERTY,
                 DAMAGE_CROPS
            FROM details 
           WHERE STATE = 'LOUISIANA'
        ORDER BY CZ_NAME,
                 YEARMONTH,
                 DAY;
        '''

'''
Possible columns of interest: 

,
MAGNITUDE,
MAGNITUDE_TYPE,
FLOOD_CAUSE
'''

results = c.execute(query)
df = pd.DataFrame(results.fetchall())

# Change column names to lowercase
names = [description[0].lower() for description in results.description]
df.columns = names

conn.close()

In [183]:
# Importing FIPS data from storm events database
conn = sqlite3.connect('../data/raw/storms.db')
c = conn.cursor()

query = '''
          SELECT STATE_FIPS,
                 CZ_FIPS,
                 CZ_NAME AS PARISH,
                 BEGIN_LAT AS LAT,
                 BEGIN_LON AS LON
            FROM details 
           WHERE STATE = 'LOUISIANA'
        ORDER BY CZ_FIPS
        '''

results = c.execute(query)
fips_df = pd.DataFrame(results.fetchall())

# Change column names to lowercase
names = [description[0].lower() for description in results.description]
fips_df.columns = names

conn.close()

In [184]:
def separate(yearmonth: int):
    '''
    Separates the yearmonth column into their own respective columns
    Return year (int) and month (int)
    '''
    yearmonth = str(yearmonth)
    return int(yearmonth[:4]), int(yearmonth[-2:])


df[['year', 'month']] = pd.DataFrame(df.yearmonth.apply(separate).tolist(), index=df.index)
df = df.drop('yearmonth', axis=1)
df['year'] = df.year.astype(int)
df['month'] = df.month.astype(int)

# Convert separate year, month, day columns into datetime
df['date'] = pd.to_datetime(df.year*10000 + df.month*100 + df.day, format='%Y%m%d')
df = df.drop(['year', 'month', 'day'], axis=1)

# Change order of DataFrame
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

In [185]:
df.head()

,date,parish,event_type,lat,lon,injuries_direct,injuries_indirect,deaths_direct,deaths_indirect,damage_property,damage_crops
0,1957-03-03,None,Thunderstorm Wind,30.50,-92.18,0,0,0,0,0,0
1,1959-06-08,None,Thunderstorm Wind,29.68,-90.18,0,0,0,0,0,0
2,1959-09-21,None,Thunderstorm Wind,32.28,-93.40,0,0,0,0,0,0
3,1960-05-06,None,Thunderstorm Wind,30.08,-90.18,0,0,0,0,0,0
4,1961-03-20,None,Thunderstorm Wind,29.00,-89.40,0,0,0,0,0,0


In [186]:
# Capitalize first letter of each part of name
df['parish'] = df.parish.str.title()
fips_df['parish'] = fips_df.parish.str.title()

In [187]:
df.parish.unique()

array([None, 'Acadia', 'Allen', 'Ascension', 'Assumption', 'Avoyelles',
       'Beauregard', 'Bienville', 'Bossier', 'Caddo', 'Calcasieu',
       'Caldwell', 'Cameron', 'Catahoula', 'Claiborne', 'Concordia',
       'De Soto', 'East Baton Rouge', 'East Cameron', 'East Carroll',
       'East Feliciana', 'Evangeline', 'Franklin', 'Grant', 'Iberia',
       'Iberville', 'Jackson', 'Jefferson', 'Jefferson Davis', 'La Salle',
       'Lafayette', 'Lafourche', 'Laz038>040 - 056>070', 'Laz067 - 070',
       'Lincoln', 'Livingston', 'Lower Jefferson', 'Lower Lafourche',
       'Lower Plaquemines', 'Lower St. Bernard', 'Lower St. Martin',
       'Lower Terrebonne', 'Madison', 'Morehouse', 'Natchitoches',
       'Northern Tangipahoa', 'Orleans', 'Ouachita', 'Plaquemines',
       'Pointe Coupee', 'Rapdies', 'Rapides', 'Red River', 'Richland',
       'Sabine', 'Sabine And Natchitoches', 'Southern Tangipahoa',
       'St. Bernard', 'St. Charles', 'St. Helena', 'St. James',
       'St. John The Baptist

In [188]:
# Inspecting odd cz_name
print(df[df.parish == 'Laz038>040 - 056>070'])
print()
print(df[df.parish == 'Laz067 - 070'])

            date                parish event_type  lat  lon  injuries_direct  \
12110 1995-10-27  Laz038>040 - 056>070       Hail  NaN  NaN                0   

       injuries_indirect  deaths_direct  deaths_indirect damage_property  \
12110                  0              0                0               0   

      damage_crops  
12110            0  

            date        parish event_type  lat  lon  injuries_direct  \
12111 1994-10-20  Laz067 - 070       Hail  NaN  NaN                0   

       injuries_indirect  deaths_direct  deaths_indirect damage_property  \
12111                  0              0                0               0   

      damage_crops  
12111            0  


In [189]:
# Drop rows with odd cz_name
df = df[(df.parish != 'Laz038>040 - 056>070') & (df.parish != 'Laz067 - 070')]

In [190]:
# Fix typo
df['parish'] = df.parish.replace({'Rapdies': 'Rapides', 'St. John The Baptist': 'St. John the Baptist'})

In [191]:
# Find the FIPS code for cz_name that aren't actual parishes
check_fips = ['East Cameron', 'Lower Jefferson', 'Lower Lafourche', 'Lower Plaquemines', 'Lower St. Bernard', 
              'Lower St. Martin', 'Lower Terrebonne', 'Northern Tangipahoa', 'Sabine And Natchitoches', 
              'Southern Tangipahoa', 'Upper Jefferson', 'Upper Lafourche', 'Upper Plaquemines', 'Upper St. Bernard', 
              'Upper St. Martin', 'Upper Terrebonne', 'West Cameron']

for parish in check_fips:
    print(parish + ': ' + str(fips_df[fips_df.parish == parish]['cz_fips'].unique()) + '\n')

East Cameron: [74]

Lower Jefferson: [68]

Lower Lafourche: [67]

Lower Plaquemines: [69]

Lower St. Bernard: [70]

Lower St. Martin: [55]

Lower Terrebonne: [66]

Northern Tangipahoa: [71]

Sabine And Natchitoches: [85]

Southern Tangipahoa: [72]

Upper Jefferson: [61]

Upper Lafourche: [59]

Upper Plaquemines: [63]

Upper St. Bernard: [64]

Upper St. Martin: [45]

Upper Terrebonne: [65]

West Cameron: [73]



In [192]:
# Compare above FIPS with actual parish FIPS
parish_fips = ['Cameron', 'Jefferson', 'Lafourche', 'Plaquemines', 'St. Bernard', 'St. Martin', 'Terrebonne', 
               'Tangipahoa', 'Sabine', 'Natchitoches']

for parish in parish_fips:
    print(parish + ': ' + str(fips_df[fips_df.parish == parish]['cz_fips'].unique()) + '\n')

Cameron: [23 51]

Jefferson: [51]

Lafourche: [57]

Plaquemines: [75]

St. Bernard: [87]

St. Martin: [99]

Terrebonne: [109]

Tangipahoa: [ 38 105]

Sabine: [17 85]

Natchitoches: [18 69]



In [193]:
# Get coordinates (if any) for cz_name that aren't actual parishes
for parish in check_fips:
    print(df[(df.parish == parish) & (df.lat.notnull())].head(1))
    print()

Empty DataFrame
Columns: [date, parish, event_type, lat, lon, injuries_direct, injuries_indirect, deaths_direct, deaths_indirect, damage_property, damage_crops]
Index: []

Empty DataFrame
Columns: [date, parish, event_type, lat, lon, injuries_direct, injuries_indirect, deaths_direct, deaths_indirect, damage_property, damage_crops]
Index: []

Empty DataFrame
Columns: [date, parish, event_type, lat, lon, injuries_direct, injuries_indirect, deaths_direct, deaths_indirect, damage_property, damage_crops]
Index: []

Empty DataFrame
Columns: [date, parish, event_type, lat, lon, injuries_direct, injuries_indirect, deaths_direct, deaths_indirect, damage_property, damage_crops]
Index: []

Empty DataFrame
Columns: [date, parish, event_type, lat, lon, injuries_direct, injuries_indirect, deaths_direct, deaths_indirect, damage_property, damage_crops]
Index: []

Empty DataFrame
Columns: [date, parish, event_type, lat, lon, injuries_direct, injuries_indirect, deaths_direct, deaths_indirect, damage_pro

In [194]:
# Only one - Sabine and Natchitoches - use coordinates to find parish
url = 'http://www.datasciencetoolkit.org/coordinates2politics/'
coord = '31.62%2c-93.43'
r = requests.get(url + coord)
print(r.json())

[{'politics': [{'type': 'admin2', 'friendly_type': 'country', 'name': 'United States', 'code': 'usa'}, {'type': 'admin4', 'friendly_type': 'state', 'name': 'Louisiana', 'code': 'us22'}, {'type': 'constituency', 'friendly_type': 'constituency', 'name': 'Fourth district, LA', 'code': '22_04'}, {'type': 'admin6', 'friendly_type': 'county', 'name': 'Sabine', 'code': '22_085'}], 'location': {'latitude': 31.62, 'longitude': -93.43}}]


In [195]:
# Create dummy DataFrame to see if these FIPS codes are found
fake = [['22074', 1], ['22068', 2], ['22070', 3], ['22066', 4], ['22072', 5], ['22064', 6], ['22038', 7], ['22018', 8]]
fdf = pd.DataFrame(fake, columns=['fips', 'data'])

In [196]:
values = fdf['data'].tolist()
fips = fdf['fips'].tolist()

endpts = list(np.mgrid[min(values):max(values):4j])
fig = ff.create_choropleth(fips=fips, values=values, scope=['Louisiana'], show_state_data=True,
                           binning_endpoints=endpts, round_legend_values=True,
                           plot_bgcolor='rgb(229,229,229)',
                           paper_bgcolor='rgb(229,229,229)',
                           county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
                           exponent_format=True)
fig.layout.template = None
fig.show()

/Users/jenny/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


/Users/jenny/anaconda3/lib/python3.7/site-packages/plotly/figure_factory/_county_choropleth.py:847: UserWarning:

Unrecognized FIPS Values

Whoops! It looks like you are trying to pass at least one FIPS value that is not in our shapefile of FIPS and data for the counties. Your choropleth will still show up but these counties cannot be shown.
Unrecognized FIPS are: [22074, 22068, 22070, 22066, 22072, 22064, 22038, 22018]



In [197]:
# None of the FIPS codes are valid, so change to parish
parish_dict = {'East Cameron': 'Cameron', 'Lower Jefferson': 'Jefferson', 'Lower Lafourche': 'Lafourche', 
               'Lower Plaquemines': 'Plaquemines', 'Lower St. Bernard': 'St. Bernard', 
               'Lower St. Martin': 'St. Martin', 'Lower Terrebonne': 'Terrebonne', 'Northern Tangipahoa': 'Tangipahoa',
               'Sabine And Natchitoches': 'Sabine', 'Southern Tangipahoa': 'Tangipahoa', 
               'Upper Jefferson': 'Jefferson', 'Upper Lafourche': 'Lafourche', 'Upper Plaquemines': 'Plaquemines', 
               'Upper St. Bernard': 'St. Bernard', 'Upper St. Martin': 'St. Martin', 'Upper Terrebonne': 'Terrebonne',
               'West Cameron': 'Cameron'}
df['parish'] = df.parish.replace(parish_dict)

In [198]:
df[df.parish.isnull()]

,date,parish,event_type,lat,lon,injuries_direct,injuries_indirect,deaths_direct,deaths_indirect,damage_property,damage_crops
0,1957-03-03,None,Thunderstorm Wind,30.50,-92.18,0,0,0,0,0,0
1,1959-06-08,None,Thunderstorm Wind,29.68,-90.18,0,0,0,0,0,0
2,1959-09-21,None,Thunderstorm Wind,32.28,-93.40,0,0,0,0,0,0
3,1960-05-06,None,Thunderstorm Wind,30.08,-90.18,0,0,0,0,0,0
4,1961-03-20,None,Thunderstorm Wind,29.00,-89.40,0,0,0,0,0,0
5,1962-01-09,None,Thunderstorm Wind,29.00,-89.40,0,0,0,0,0,0
6,1963-07-01,None,Thunderstorm Wind,29.48,-91.68,0,0,0,0,0,0
7,1963-11-09,None,Thunderstorm Wind,29.00,-89.40,0,0,0,0,0,0
8,1964-02-24,None,Thunderstorm Wind,29.20,-90.18,0,0,0,0,0,0
9,1964-04-25,None,Thunderstorm Wind,30.50,-92.00,0,0,0,0,0,0


In [199]:
def find_parish(lat: float, lon: float, debug=False):
    '''
    Use coordinates2politics API to find parish based on coordinates
    Returns parish name (str) or None
    '''
    url = 'http://www.datasciencetoolkit.org/coordinates2politics/'
    coord = str(lat) + '%2c' + str(lon)
    r = requests.get(url + coord)
    try:
        # Print coordinates returned by API
        if debug:
            print(i)
            print(r.json()[0]['location'])
        # Iterate through each dict
        for d in r.json()[0]['politics']:
            # Add parish name if found
            if d['friendly_type'] == 'county':
                if debug:
                    print(d['name'])
                    print(d['code'])
                return d['name']
    # NoneType when API returns no politics data
    except TypeError:
        if debug:
            print('No data')
        return None


no_parish_count = 0

for i, row in df[df.parish.isnull()].iterrows():
    parish = find_parish(row['lat'], row['lon'])
    if parish:
        df.loc[df.index == i, 'parish'] = parish
    else:
        no_parish_count += 1
    
print('Couldn\'t fix count:', no_parish_count)

Couldn't fix count: 14


In [200]:
print(len(df[df.parish.isnull()]))
df[df.parish.isnull()]

14


,date,parish,event_type,lat,lon,injuries_direct,injuries_indirect,deaths_direct,deaths_indirect,damage_property,damage_crops
4,1961-03-20,None,Thunderstorm Wind,29.00,-89.40,0,0,0,0,0,0
5,1962-01-09,None,Thunderstorm Wind,29.00,-89.40,0,0,0,0,0,0
6,1963-07-01,None,Thunderstorm Wind,29.48,-91.68,0,0,0,0,0,0
7,1963-11-09,None,Thunderstorm Wind,29.00,-89.40,0,0,0,0,0,0
13,1964-08-05,None,Thunderstorm Wind,29.38,-91.28,0,0,0,0,0,0
25,1969-02-14,None,Thunderstorm Wind,29.28,-92.40,0,0,0,0,0,0
28,1969-05-07,None,Thunderstorm Wind,29.38,-91.28,0,0,0,0,0,0
30,1969-11-13,None,Thunderstorm Wind,28.60,-90.28,0,0,0,0,0,0
31,1970-01-05,None,Thunderstorm Wind,29.08,-90.18,0,0,0,0,0,0
39,1973-03-04,None,Hail,29.23,-93.33,0,0,0,0,0,0


In [201]:
# Drop remaining nulls
df = df[df.parish.notnull()].reset_index(drop=True)

In [202]:
# Add column with FIPS code
parish_fips_dict = {'Acadia': '22001',
                    'Allen': '22003',
                    'Ascension': '22005',
                    'Assumption': '22007',
                    'Avoyelles': '22009',
                    'Beauregard': '22011',
                    'Bienville': '22013',
                    'Bossier': '22015',
                    'Caddo': '22017',
                    'Calcasieu': '22019',
                    'Caldwell': '22021',
                    'Cameron': '22023',
                    'Catahoula': '22025',
                    'Claiborne': '22027',
                    'Concordia': '22029',
                    'De Soto': '22031',
                    'East Baton Rouge': '22033',
                    'East Carroll': '22035', 
                    'East Feliciana': '22037',
                    'Evangeline': '22039',
                    'Franklin': '22041',
                    'Grant': '22043',
                    'Iberia': '22045', 
                    'Iberville': '22047',
                    'Jackson': '22049',
                    'Jefferson': '22051',
                    'Jefferson Davis': '22053',
                    'La Salle': '22059', 
                    'Lafayette': '22055',
                    'Lafourche': '22057',
                    'Lincoln': '22061', 
                    'Livingston': '22063',
                    'Madison': '22065', 
                    'Morehouse': '22067', 
                    'Natchitoches': '22069', 
                    'Orleans': '22071', 
                    'Ouachita': '22073', 
                    'Plaquemines': '22075', 
                    'Pointe Coupee': '22077', 
                    'Rapides': '22079', 
                    'Red River': '22081', 
                    'Richland': '22083', 
                    'Sabine': '22085',
                    'St. Bernard': '22087',
                    'St. Charles': '22089', 
                    'St. Helena': '22091',
                    'St. James': '22093',
                    'St. John the Baptist': '22095',
                    'St. Landry': '22097', 
                    'St. Martin': '22099', 
                    'St. Mary': '22101', 
                    'St. Tammany': '22103',
                    'Tangipahoa': '22105', 
                    'Tensas': '22107', 
                    'Terrebonne': '22109', 
                    'Union': '22111', 
                    'Vermilion': '22113', 
                    'Vernon': '22115',
                    'Washington': '22117', 
                    'Webster': '22119', 
                    'West Baton Rouge': '22121', 
                    'West Carroll': '22123', 
                    'West Feliciana': '22125', 
                    'Winn': '22127'}
df['fips'] = df.parish.map(parish_fips_dict)

# Drop unnecessary columns
df = df.drop(['lat', 'lon'], axis=1)

In [203]:
# Check if any null FIPS
df[df.fips.isnull()]

,date,parish,event_type,injuries_direct,injuries_indirect,deaths_direct,deaths_indirect,damage_property,damage_crops,fips
10,1965-01-22,Wilkinson,Thunderstorm Wind,0,0,0,0,0,0,NaN
20,1969-02-27,Newton,Hail,0,0,0,0,0,0,NaN
26,1971-06-23,Adams,Thunderstorm Wind,0,0,0,0,0,0,NaN
27,1971-12-06,Jasper,Thunderstorm Wind,0,0,0,0,0,0,NaN
30,1973-03-15,Harrison,Hail,0,0,0,0,0,0,NaN


In [204]:
# Drop null FIPS rows
drop_rows = ['Wilkinson', 'Newton', 'Adams', 'Jasper', 'Harrison']
df = df[~df.parish.isin(drop_rows)]

In [205]:
df.head()

,date,parish,event_type,injuries_direct,injuries_indirect,deaths_direct,deaths_indirect,damage_property,damage_crops,fips
0,1957-03-03,St. Landry,Thunderstorm Wind,0,0,0,0,0,0,22097
1,1959-06-08,Lafourche,Thunderstorm Wind,0,0,0,0,0,0,22057
2,1959-09-21,Bienville,Thunderstorm Wind,0,0,0,0,0,0,22013
3,1960-05-06,Jefferson,Thunderstorm Wind,0,0,0,0,0,0,22051
4,1964-02-24,Lafourche,Thunderstorm Wind,0,0,0,0,0,0,22057


In [208]:
df[df.index == 59]

,date,parish,event_type,injuries_direct,injuries_indirect,deaths_direct,deaths_indirect,damage_property,damage_crops,fips
59,1973-09-11,Acadia,Tornado,0,0,0,0,.25K,0,22001


In [158]:
df['event_type'].unique()

array(['Thunderstorm Wind', 'Hail', 'Tornado', 'Drought', 'Flash Flood',
       'Flood', 'Lightning', 'Tropical Storm', 'Heat', 'High Wind',
       'Hurricane (Typhoon)', 'Strong Wind', 'Storm Surge/Tide',
       'Winter Weather', 'Heavy Rain', 'Funnel Cloud', 'Cold/Wind Chill',
       'Ice Storm', 'Winter Storm', 'Coastal Flood',
       'Tropical Depression', 'Heavy Snow', 'Hurricane', 'Freezing Fog',
       'Excessive Heat', 'Wildfire', 'Dense Fog', 'Astronomical Low Tide',
       'Sleet', 'Seiche', 'Extreme Cold/Wind Chill', 'Dust Devil',
       'Waterspout', 'Rip Current'], dtype=object)

In [159]:
df[df.event_type == 'Strong Wind']

,date,parish,event_type,injuries_direct,injuries_indirect,deaths_direct,deaths_indirect,damage_property,damage_crops,fips
250,2008-03-03,Acadia,Strong Wind,0,0,0,0,10.00K,0.00K,22001
744,2006-03-09,Ascension,Strong Wind,0,0,0,0,10K,None,22005
1111,2008-03-18,Avoyelles,Strong Wind,0,0,0,0,10.00K,0.00K,22009
1501,2008-03-18,Beauregard,Strong Wind,0,0,0,0,15.00K,0.00K,22011
1919,2008-08-03,Bienville,Strong Wind,0,0,0,0,4.00K,0.00K,22013
2754,2008-03-18,Bossier,Strong Wind,0,0,0,0,50.00K,0.00K,22015
2777,2008-12-27,Bossier,Strong Wind,0,0,0,0,3.00K,0.00K,22015
2874,2011-02-24,Bossier,Strong Wind,0,0,0,0,5.00K,0.00K,22015
2885,2011-04-25,Bossier,Strong Wind,0,0,0,0,20.00K,0.00K,22015
3054,2017-01-10,Bossier,Strong Wind,1,0,1,0,0.00K,0.00K,22015


In [225]:
def format_damage(damage: str):
    '''
    damage_property and damage_crops are str values appended with K
    Formats to proper float values
    Returns damage (float)
    '''
    if damage is '0' or not damage:
        damage = 0
    elif damage is 'K':
        damage = 0
    else:
        damage = damage[:-1]
    return float(damage) * 1000


df['damage_property'] = df.damage_property.apply(format_damage)
df['damage_crops'] = df.damage_crops.apply(format_damage)
df.head()

,date,parish,event_type,injuries_direct,injuries_indirect,deaths_direct,deaths_indirect,damage_property,damage_crops,fips
0,1957-03-03,St. Landry,Thunderstorm Wind,0,0,0,0,0.0,0.0,22097
1,1959-06-08,Lafourche,Thunderstorm Wind,0,0,0,0,0.0,0.0,22057
2,1959-09-21,Bienville,Thunderstorm Wind,0,0,0,0,0.0,0.0,22013
3,1960-05-06,Jefferson,Thunderstorm Wind,0,0,0,0,0.0,0.0,22051
4,1964-02-24,Lafourche,Thunderstorm Wind,0,0,0,0,0.0,0.0,22057


In [67]:
df2 = df.copy()
df2.index = df2.date
df2 = df2.drop('date', axis=1)
df2.head()

,cz_name,event_type,fips
date,,,
1957-03-03,St. Landry,Thunderstorm Wind,22097
1959-06-08,Lafourche,Thunderstorm Wind,22057
1959-09-21,Bienville,Thunderstorm Wind,22013
1960-05-06,Jefferson,Thunderstorm Wind,22051
1964-02-24,Lafourche,Thunderstorm Wind,22057


In [72]:
df2.groupby(['event_type', 'fips']).count()

cz_name
event_type            fips          
Astronomical Low Tide 22019        3
                      22023       69
                      22045        6
                      22101       10
                      22113       33
Coastal Flood         22001        1
                      22019        1
                      22023       18
                      22051        5
                      22057        3
                      22063        1
                      22071        3
                      22075        1
                      22087        4
                      22089        1
                      22095        2
                      22101        7
                      22103        5
                      22105        1
                      22109        5
                      22113        6
Cold/Wind Chill       22001        1
                      22003        2
                      22005        2
                      22007        2
                      22009        1
                      22011        1
                      22013        2
                      22015        2
                      22017        2
...                              ...
Winter Weather        22063        2
                      22065        2
                      22067        2
                      22069       12
                      22071        2
                      22073        9
                      22077        2
                      22079       15
                      22081        8
                      22085       12
                      22087        1
                      22089        1
                      22091        1
                      22093        1
                      22095        3
                      22097        7
                      22099       14
                      22101        6
                      22103        2
                      22105        3
                      22107        1
                      22109        2
                      22111       13
                      22113       10
                      22115       12
                      22119        8
                      22121        1
                      22123        2
                      22125        3
                      22127       11

[1205 rows x 1 columns]

In [35]:
test = df[['date', 'fips', 'event_type']]
test.index = test.date
test = test.drop('date', axis=1)

In [39]:
test2 = test.groupby('fips').count()
test2.head()

,event_type
fips,
22001,330
22003,277
22005,164
22007,96
22009,307


In [42]:
test2.describe()

,event_type
count,64.000000
mean,381.734375
std,240.583566
min,96.000000
25%,242.250000
50%,326.000000
75%,458.250000
max,1640.000000


In [58]:
values = test2['event_type'].tolist()
fips = test2.index.tolist()

endpts = list(np.mgrid[min(values):max(values):4j])
colorscale = [
    'rgb(68.0, 1.0, 84.0)',
    'rgb(66.0, 64.0, 134.0)',
    'rgb(38.0, 130.0, 142.0)',
    'rgb(63.0, 188.0, 115.0)',
    'rgb(216.0, 226.0, 25.0)'
]
fig = ff.create_choropleth(
    fips=fips, values=values, scope=['Louisiana'], show_state_data=True,
    colorscale=colorscale, binning_endpoints=endpts, round_legend_values=True,
    plot_bgcolor='rgb(229,229,229)',
    paper_bgcolor='rgb(229,229,229)',
    legend_title='Population by County',
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    exponent_format=True,
)
fig.layout.template = None
fig.show()